# Deep learning implementation

Single Shot Detector based Keras

In [ ]:
!pip install efficientnet
!wget -P checkpoints https://github.com/500swapnil/Keras_Efficientnet_SSD/releases/download/v1.0/efficientnetb0_SSD.h5
!wget -P checkpoints https://github.com/500swapnil/Keras_Efficientnet_SSD/releases/download/v1.0/efficientnetb5_SSD.h5

## Single Shot Detector

### predict method

In [ ]:
# load python packet   
# ----------------------start--------------------------------
# load necessary packets
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.image import imread
import tensorflow as tf

# load utilities
from utils.priors import *
from model.ssd import ssd
from utils.post_processing import post_process
from utils.preprocessing import prepare_for_prediction
# -------------------------end-------------------------------

In [ ]:
# define parameters & directory paths
# ----------------------start--------------------------------
# define parameter
IMAGE_SIZE = [300, 300]
# define parameter
BATCH_SIZE = 16
# choose backbone net [B0, B1, B2, ..., B7]
MODEL_NAME = 'B0'

# directory path of weight of pretrained model [./checkpoints/efficientnetb0_SSD.h5]
checkpoint_filepath = ??? 
# directory path of input image
INPUT_DIR = './images'
# directory path of output image
OUTPUT_DIR = './outputs'
# -------------------------end-------------------------------

In [ ]:
# define parameters of SSD
# ----------------------start--------------------------------
# set positive box's threshold
iou_threshold = 0.5
# set anchor center's variance
center_variance = 0.1
# set anchor size's variance
size_variance = 0.2

# set anchor boxes
specs = [
                SSDSpec(38, 8, SSDBoxSizes(30, 60), [2]),
                SSDSpec(19, 16, SSDBoxSizes(60, 111), [2, 3]),
                SSDSpec(10, 32, SSDBoxSizes(111, 162), [2, 3]),
                SSDSpec(5, 64, SSDBoxSizes(162, 213), [2, 3]),
                SSDSpec(3, 100, SSDBoxSizes(213, 264), [2]),
                SSDSpec(1, 300, SSDBoxSizes(264, 315), [2])
        ]

# create SSD's anchor boxes
priors = generate_ssd_priors(specs, IMAGE_SIZE[0])
target_transform = MatchPrior(priors, center_variance, size_variance, iou_threshold)
# -------------------------end-------------------------------

In [ ]:
# build SSD & load checkpoint
# ----------------------start--------------------------------
# build SSD
print("Building SSD Model with EfficientNet{0} backbone..".format(MODEL_NAME))
model = ssd(MODEL_NAME, pretrained=False)

# load checkpoint
print("Loading Checkpoint..")
if checkpoint_filepath is not None:
    print("Loading Checkpoint..", checkpoint_filepath)
    model.load_weights(checkpoint_filepath)
else:
    print("Training from with only base model pretrained on imagenet")
# -------------------------end-------------------------------

In [ ]:
# load image
# ----------------------start--------------------------------
dataset = tf.data.Dataset.list_files(INPUT_DIR + '/*', shuffle=False)
filenames = list(dataset.as_numpy_iterator())
dataset = dataset.map(prepare_for_prediction)
dataset = dataset.batch(BATCH_SIZE)
# -------------------------end-------------------------------

In [ ]:
# predict the image
# ----------------------start--------------------------------
pred = model.predict(dataset, verbose=1)
predictions = post_process(pred, target_transform, confidence_threshold=0.4)
# -------------------------end-------------------------------

In [ ]:
# draw boxes of human & save image
# ----------------------start--------------------------------
print("Prediction Complete")
for i, path in enumerate(filenames):
    path_string = path.decode("utf-8")
    im = imread(path_string)
    filename = path_string.split('/')[-1]
    fig, ax = plt.subplots(1, figsize=(15, 15))
    ax.imshow(im)
    pred_boxes, pred_scores, pred_labels = predictions[i]
    if pred_boxes.size > 0:
        draw_bboxes(pred_boxes, ax , labels=pred_labels, IMAGE_SIZE=im.shape[:2])
    plt.axis('off')
    plt.savefig(os.path.join(OUTPUT_DIR, 'out_'+ filename), bbox_inches='tight', pad_inches=0)
    
print("Output is saved in", OUTPUT_DIR)
# -------------------------end-------------------------------

### try to predict without training

```bash
# predict with new model
!python predict_ssd.py

# predict with final model
!python predict_ssd.py --checkpoint ./checkpoints/efficientnetb0_SSD.h5
```

In [ ]:
!python predict_ssd.py

### try to predict with training weigth

In [ ]:
!python predict_ssd.py --checkpoint ./checkpoints/efficientnetb0_SSD.h5

### train method

In [ ]:
# load python packet   
# ----------------------start--------------------------------
# load necessary packets
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.image import imread
import tensorflow as tf
import tensorflow_datasets as tfds

# load utilities
from utils.priors import *
from model.ssd import ssd
from model.loss import multibox_loss
from utils.preprocessing import prepare_dataset
from tensorflow.keras.callbacks import LearningRateScheduler,  ReduceLROnPlateau, ModelCheckpoint
# -------------------------end-------------------------------

In [ ]:
# define parameters & directory paths
# ----------------------start--------------------------------
# define directory path of dataset
DATASET_DIR = './dataset'
# define parameter
IMAGE_SIZE = [300, 300]
# define parameter
BATCH_SIZE = 16
# choose backbone net [B0, B1, B2, ..., B7]
MODEL_NAME = 'B0'
# define number of epochs
EPOCHS = 5

# directory path of weight of pretrained model [./checkpoints/efficientnetb0_SSD.h5]
checkpoint_filepath = ??? 
# -------------------------end-------------------------------

In [ ]:
# build data loader
# ----------------------start--------------------------------
print("Loading Data..")
train_data = tfds.load("voc", data_dir=DATASET_DIR, split='train')
number_train = train_data.reduce(0, lambda x, _: x + 1).numpy()
print("Number of Training Files:", number_train)
# -------------------------end-------------------------------

In [ ]:
# define parameters of SSD
# ----------------------start--------------------------------
# set positive box's threshold
iou_threshold = 0.5
# set anchor center's variance
center_variance = 0.1
# set anchor size's variance
size_variance = 0.2

# set anchor boxes
specs = [
                SSDSpec(38, 8, SSDBoxSizes(30, 60), [2]),
                SSDSpec(19, 16, SSDBoxSizes(60, 111), [2, 3]),
                SSDSpec(10, 32, SSDBoxSizes(111, 162), [2, 3]),
                SSDSpec(5, 64, SSDBoxSizes(162, 213), [2, 3]),
                SSDSpec(3, 100, SSDBoxSizes(213, 264), [2]),
                SSDSpec(1, 300, SSDBoxSizes(264, 315), [2])
        ]

# create SSD's anchor boxes
priors = generate_ssd_priors(specs, IMAGE_SIZE[0])
target_transform = MatchPrior(priors, center_variance, size_variance, iou_threshold)
# instantiate the datasets
training_dataset = prepare_dataset(train_data, IMAGE_SIZE, BATCH_SIZE, target_transform, train=True)
# -------------------------end-------------------------------

In [ ]:
# build SSD & calculate number of steps
# ----------------------start--------------------------------
# build SSD
print("Building SSD Model with EfficientNet{0} backbone..".format(MODEL_NAME))
model = ssd(MODEL_NAME)
# calculate number of steps
steps_per_epoch = number_train // BATCH_SIZE
print("Number of Train Batches:", steps_per_epoch)
# method summarizes detail of model
model.summary()
# -------------------------end-------------------------------

In [ ]:
# define parameters of model
# ----------------------start--------------------------------
# define learning rate scheduler
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=1, min_lr=1e-5, verbose=1)
# define checkpoint method
checkpoint = ModelCheckpoint(checkpoint_filepath, monitor='val_loss', save_best_only=True, save_weights_only=True, verbose=1)
# set initial learning rate
base_lr = 1e-3 if checkpoint_filepath is None else 1e-5
# -------------------------end-------------------------------

In [ ]:
# define compiler
# ----------------------start--------------------------------
model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate=base_lr),
    loss = multibox_loss
)
# -------------------------end-------------------------------

In [ ]:
# resume checkpoint
# ----------------------start--------------------------------
if checkpoint_filepath is not None:
    print("Continuing Training from", checkpoint_filepath)
    model.load_weights(checkpoint_filepath)
else:
    print("Training from with only base model pretrained on imagenet")
# -------------------------end-------------------------------

In [ ]:
# train model
# ----------------------start--------------------------------
history = model.fit(training_dataset, 
                    steps_per_epoch=steps_per_epoch, 
                    epochs=EPOCHS,
                    callbacks=[reduce_lr,checkpoint]) 
# -------------------------end-------------------------------

### inference method
```bash
# run inference method with new model
!python inference_ssd.py -debug -fill --checkpoint None

# run inference method with final model
!python inference_ssd.py -debug -fill --checkpoint ./checkpoints/efficientnetb0_SSD.h5
```